# Implicit References

Previously, we looked at explicit references in Lettuce using statements such as `NewRef`, `DeRef` and
`AssignRef`.

As a warmup exercise, what value does the following program compute?

~~~ 
let x = NewRef(10) in 
   let g = function (y) 
            DeRef(x)
        in 
       let dummy = AssignRef(x, 20) in 
            g (dummy)
~~~

*Answer:* Should be 20. Why? Note that x maps itself to a location (1) in the memory which stores the value 10.
The function g ignores its argument and returns the dereference of x, i.e, whatever the location (1) points to.
Next, we change this location to 20 under the hood. So when g is finally called, we get the contents of location (1), which is 20.


Thus far, we have been using explicit references, wherein, we have to use NewRef to create a new reference,
use DeRef everytime we want its value and AssignRef to assign to it.

If we forget to use DeRef, we do not get the same result. As an example, consider the program

~~~
let x = NewRef(1) in
   x
~~~

It returns a reference to a location in the store.

However,  the program

~~~
let x = NewRef(1) in 
   DeRef(x)
~~~

returns the value 1.

Our goal in this notebook is to mimic the behavior of mutable vars in scala. 

In [2]:
var x = 10 // We do not need special NewRef when a var is created
val y = x // Note that no deref is needed to get the value of x
x = x + 1 // Here, we can assign to a var just like a ref, but on the RHS, we did notneed a deref.
print(s"The value of x is $x")

The value of x is 11

x: Int = 11
y: Int = 10

In this lecture, we will study **implicit references**. 
- These are references but we do not want to use NewRef to create a reference, we will look at syntax similar to `var` declarations in Scala.
- We do not wish to use derefs to get the value. Whenever we refer to a var, we would directly like to get its value without using a deref.
- Finally, we would like to use assignment on these vars just like we did on references.

### Syntax for Vars in Lettuce

Let us add an extra bit of syntax that will allow us to create such implicit references. Since this is lettuce,
we will use the `let var` binding to specify that whatever is being bound to will be an implicit reference.

~~~
let var x = <expr> in 
      <body expr> 
~~~

We will discard explicit reference operations `NewRef`, `AssignRef`, `DeRef`. Instead, we will just have the
`let var` binding to create new implicit references (we will call them "vars" since they will behave exactly like
scala's vars), and assignments.


Let us do some example programs.

~~~
let var x = 10 in 
    let dummy = AssignVar(x, 20) in 
        x
~~~

This will be equivalent to 

~~~
let x = NewRef(10) in 
    let dummy = AssignRef(x, 20) in 
        DeRef(x)
~~~

Note that `AssignRef` is now  called `AssignVar`, there is no more `DeRef`. These are the two major changes to our syntax.

What does this program do?

~~~ 
let var x = 10 in 
   let g = function (y) 
            x
        in 
       let dummy = AssignVar(x, 20) in 
            g (dummy)
~~~

We can implement some funky stuff such as changing the binding of a function.

~~~
let var f = function (x) x + 10 in 
  let g = function (y) y - 10 in 
    let d = f(10) in 
       let dummy = AssignVar(f, g) in 
          d - f(10)
~~~

Notice how we assigned `f` to `g` because `f` is a var. So it is assignable. Also note that we do not need
to say DeRef in this language. Finally, notice how g is an immutable val whereas f is a mutable var.

## Abstract Syntax of Lettuce with Mutable References

$$\begin{array}{rcll}
\mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
\mathbf{Expr} & \rightarrow & Const(\mathbf{Number}) \\
 & | & Ident(\mathbf{Identifier}) \\
 & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Minus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Mult (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Geq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Eq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & IfThenElse(\mathbf{Expr}, \mathbf{Expr}, \mathbf{Expr}) & \text{if (expr) then expr else expr} \\
 & | & Let( \mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr}) & \text{let identifier = expr in expr} \\
 & | & FunDef( \mathbf{Identifier}, \mathbf{Expr}) & \text{function (identifier-formal-parameter) expr } \\ 
 & | & FunCall(\mathbf{Expr}, \mathbf{Expr}) & \text{function call - expr(expr)} \\
 & | & \color{red}{LetVar}(\mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr}) & \text{let var stmt -- compare to let binding.}\\
 & | & \color{red}{AssignVar}(\mathbf{Identifier}, \mathbf{Expr}) & \text{assign a var to a value. }
\end{array}$$

It should be an easy exercise to define this in Scala, which we will do later.

## Operational Semantics

How do we evaluate implicit references under the hood? Simple, the same way as we evaluate explicit references. In other words, we are going to keep the innards of our interpreter unchanged from what it was previously.

Let us recap how this was done. This part is  cut and paste from the last week's notes.

1. We will make a _new value type_ for references since our expressions an evaluate to real numbers, booleans, closures and now <font color="red"> references </font>.
2. To go hand in hand with references, we need to define an abstract notion of memory. We will call this a <font color="red"> store </font>.

<font color="red"> Note that the vars will point to these references. </font>


### Stores

Memory address are going to be numbered 0, 1, 2, .... with natural numbers and each address is going to be associated with a value. 

The store needs to support the following operations.

- Create a new memory cell in the store and assign it to a value. This is exactly what will implement the `NewRef` operation. Let us call it <font color="red">createNewCell</font> operation on stores.

- Lookup the value of a memory cell. Let us call it <font color="red"> lookupCellValue </font> operation. If the value does not exist, we will return error (and bail).

- Assign a cell to a new value. Let us call it <font color="red"> assignToCell </font>

Hand in hand with stores, we have to extend our value type. Existing value types are
- `ErrorValue` to denote error -- though in our physical implementation, we have never issued this value. We rather prefer to bail with an exception. We will continue to do so to make our lives simpler. Operational semantics will be written with error values, but the actual code will bail on error.

- `NumValue(f)` for number `f`. We will denote these as reals $\mathbb{R}$ in our semantics.
- `BoolValue(b)` for boolean `b`. We will denote these as $\mathbb{B}$ in our semantics.
- `Closure(x, e, sigma)` for closures. We will denote these as $\mathbf{Closure}$ in our semantics.

Finally, we add references:
- `Reference(j)`, which is a reference to cell number j in our store. 

### Operational Semantics

Once again, our operational semantics defines an __eval__ function that has three parts to it.
$$\newcommand\semRule[3]{\begin{array}{c} #1 \\ \hline #2 \\\end{array}\ \ \text{(#3)}} $$
$$\newcommand\eval{\mathbf{eval}}$$
$$\eval(\texttt{expr}, \texttt{env}, \texttt{store})  = (\text{value}, \text{new-store}) $$

Let us explain how it will be organized. There are two kinds of bindings: 
- Immutable vals are bound to values in the environment.
- Mutable vars are bound to `Reference(j)` where `j` is an address in the store.

### Semantic Rule for Constant

The rule for constant remains unchanged.
$$\semRule{}{\eval(\texttt{Const(f)}, \sigma, s) = ( f, s) }{const}$$

### Semantic Rule for Identifiers (How the automatic Deref is implemented)

Identifiers are going to be slightly more complex. Let us go over the interesting case first.
The following rule ensures that whenever we evaluate an identifier $x$ that happens to be
a var, we will actually return the stored value.

$$\semRule{x \in \text{domain}(\sigma),\ \sigma(x) = \texttt{Reference}(j), \texttt{lookupCell}(s, j) = v}{ \eval(\texttt{Ident(x)}, \sigma, s) = (v, s) }{ident-var-ok}$$

- The variable $x$ belongs to the environment $\sigma$.
- It evalutes to a reference to cell $j$ in store.
- Cell $j$ in the store $s$ has value $v$.
- Evaluating the identifier $x$ under environment $\sigma$ and store $s$ has value $v$.

Note how that whenever we touch a reference, we automatically chase the value corresponding to the reference
in the store and return that.

For immutable, we have the usual semantics:

$$\semRule{x \in \text{domain}(\sigma),\ \sigma(x) = v, v \not= \texttt{Reference}(j) }{ \eval(\texttt{Ident(x)}, \sigma, s) = (v, s) }{ident-val-ok}$$

### Semantic Rules for LetVar

We will now write the semantic rule for let ref. The idea is that we will generate a new reference.


$$\semRule{\eval(\texttt{e1}, \sigma, s) = (v, s_1), \;\; v \not= \mathbf{error},\;\;\; \texttt{createNewCell}(s_1,v) = (j, s_2)\;\;  }{\eval(\texttt{LetVar(x, e1, e2)}, \sigma, s) =  \eval(\texttt{e2}, \sigma[x \mapsto \texttt{Reference}(j)], s_2 )}{let-var-ok}$$

- First evaluate `e1` under the environment $\sigma$ and store $s$, results in value $v$ that is not error and store $s_1$.
- Create a new cell in $s_1$, let it be a reference to cell number j and store $s_2$.
- Evaluating `let var x = e1 in e2` is the same as evaluating `e2` under environment $\sigma[x \mapsto \texttt{Reference}(j)]$ and store $s_2$.

### Semantic Rule for AssignVar

The semantic rule for `AssignVar` is the same as that for `AssignRef` but under a new guise.

$$\semRule{x \in \text{domain}(\sigma),\ \sigma(x) = \texttt{Reference}(j),\;\;\eval(\texttt{e}, \sigma, s) = (v, s_1), \;\;  \texttt{assignToCell}(s_1, j, v) = s_2}{ \eval(\texttt{AssignVar(x, e)}, \sigma, s) = (v, s_2) } {assign-var-ok} $$
- $x$ must be mapped to a reference to cell $j$ in the current environment $\sigma$.
- $e$ must evaluate under $\sigma$ and store $s$ to $v$ with new store $s_1$.
- The store $s_2$ is obtained when we assign the value $v$ to cell $j$ in $s_1$.
- The expression $\texttt{AssignVar}(x, e)$ under env. $\sigma$ and store $s$ yields value $v$ and the store $s_2$.

The remaining rules remain unchanged from the case of explicit refs. 

### Implementing the Interpreter

In [3]:
sealed trait Program
sealed trait Expr

case class TopLevel(e: Expr) extends Program

case class Const(v: Double) extends Expr // Expr -> Const(v)
case class Ident(s: String) extends Expr // Expr -> Ident(s)

// Arithmetic Expressions
case class Plus(e1: Expr, e2: Expr) extends Expr // Expr -> Plus(Expr, Expr)
case class Minus(e1: Expr, e2: Expr) extends Expr // Expr -> Minus(Expr, Expr)
case class Mult(e1: Expr, e2: Expr) extends Expr // Expr -> Mult (Expr, Expr)

// Boolean Expressions
case class Geq(e1: Expr, e2:Expr) extends Expr
case class Eq(e1: Expr, e2: Expr) extends Expr

//If then else
case class IfThenElse(e: Expr, eIf: Expr, eElse: Expr) extends Expr

//Let bindings
case class Let(s: String, defExpr: Expr, bodyExpr: Expr) extends Expr

//Function definition
case class FunDef(param: String, bodyExpr: Expr) extends Expr

// Function call
case class FunCall(funCalled: Expr, argExpr: Expr) extends Expr

// Let Var
case class LetVar(x: String, e1: Expr, e2: Expr) extends Expr

// Assign Var
case class AssignVar(x: String, e: Expr) extends Expr

defined trait Program
defined trait Expr
defined class TopLevel
defined class Const
defined class Ident
defined class Plus
defined class Minus
defined class Mult
defined class Geq
defined class Eq
defined class IfThenElse
defined class Let
defined class FunDef
defined class FunCall
defined class LetVar
defined class AssignVar

In [4]:
// Copy from the case for explicit references
sealed trait Value


/*-- Now we can finish the rest --*/
case class NumValue(f: Double) extends Value
case class BoolValue(b: Boolean) extends Value
/*-- Note: to get recursion working, we will need to make environments different --*/
case class Closure(x: String, e: Expr, pi: Map[String, Value]) extends Value 
/* -- references are here -- */
case class Reference(j: Int) extends Value
case object ErrorValue extends Value


/*2. Operators on values */

def valueToNumber(v: Value): Double = v match {
    case NumValue(d) => d
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a number")
}

def valueToBoolean(v: Value): Boolean = v match {
    case BoolValue(b) => b
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a boolean")
}

def valueToClosure(v: Value): Closure = v match {
    case Closure(x, e, pi) => Closure(x, e, pi)
    case _ =>  throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a closure")
}

/*3. Immutable Store -- unoptimized impl. */

case class ImmutableStore(val nCells: Int, val storeMap: Map[Int, Value])
    
def createNewCell(s: ImmutableStore, v: Value): (ImmutableStore, Int) = {
        /*- make a new cell -*/
        val j = s.nCells
        val nMap = s.storeMap + (j -> v)
        val nStore = ImmutableStore(s.nCells + 1, nMap) // Make a new store with one more cell
        (nStore, j)
}
    
def lookupCellValue(s: ImmutableStore, j: Int): Value = {
        if (s.storeMap.contains(j)){
            s.storeMap(j)
        } else {
            throw new IllegalArgumentException(s"Illegal lookup of nonexistant location $j")
        }
}
    
def assignToCell(s: ImmutableStore, j: Int, v: Value): ImmutableStore = {
        if (s.storeMap.contains(j)){
            val nMap = s.storeMap + (j -> v) // Update the store map.
            ImmutableStore(s.nCells, nMap)
        } else {
            throw new IllegalArgumentException(s"Illegal assignment to nonexistent location $j")
        }
    }
    

defined trait Value
defined class NumValue
defined class BoolValue
defined class Closure
defined class Reference
defined object ErrorValue
defined function valueToNumber
defined function valueToBoolean
defined function valueToClosure
defined class ImmutableStore
defined function createNewCell
defined function lookupCellValue
defined function assignToCell

In [5]:
def evalExpr(e: Expr, env: Map[String, Value], store: ImmutableStore): (Value, ImmutableStore) = {
      /* Method to deal with binary arithmetic operations */
    
    def applyArith2 (e1: Expr, e2: Expr) (fun: (Double , Double) => Double) = {
        val (v1, store1) = evalExpr(e1, env, store)
        val (v2, store2) = evalExpr(e2, env, store1)
        val v3 = fun(valueToNumber(v1), valueToNumber(v2))
        (NumValue(v3), store2)
    }  /* -- We have deliberately curried the method --*/
    
    /* Helper method to deal with unary arithmetic */
    def applyArith1(e: Expr) (fun: Double => Double) = {
        val (v,store1) = evalExpr(e, env, store)
        val v1 = fun(valueToNumber(v))
        (NumValue(v1), store1)
    }
    
    /* Helper method to deal with comparison operators */
    def applyComp(e1: Expr, e2: Expr) (fun: (Double, Double) => Boolean) = {
        val (v1, store1) = evalExpr(e1, env, store)
        val (v2, store2) = evalExpr(e2, env, store1)
        val v3 = fun(valueToNumber(v1), valueToNumber(v2))
        (BoolValue(v3), store2)
    }
    
    e match {
        case Const(f) => (NumValue(f), store)
        
        case Ident(x) => {
            if (env contains x ) { // In scala a.b(c) can simply be written as "a b c" <macro>
                val v = env(x)
                v match {
                    case Reference(j) => { // AUTO deref
                        val v1 = lookupCellValue(store, j) // Lookup the store for address j
                        (v1, store) // return the value of reference(j) from the store.
                    }
                    case _ => (v, store) // return v and store unchanged
                }  
            } else 
                throw new IllegalArgumentException(s"Undefined identifier $x")
        }
    
    
        case Plus(e1, e2) => applyArith2 (e1, e2) ( _ + _ )
            
        case Minus(e1, e2) => applyArith2(e1, e2) ( _ - _ )
    
        case Mult(e1, e2) =>  applyArith2(e1, e2) (_ * _)
        
        case Geq(e1, e2) => applyComp(e1, e2)(_ >= _)
    
        case Eq(e1, e2) => applyComp(e1, e2)(_ == _)
        
        case IfThenElse(e1, e2, e3) => {
            val (v, store1) = evalExpr(e1, env, store)
            v match {
                case BoolValue(true) => evalExpr(e2, env, store1)
                case BoolValue(false) => evalExpr(e3, env, store1)
                case _ => throw new IllegalArgumentException(s"If-then-else condition expr: ${e1} is non-boolean -- evaluates to ${v}")
            }
        }
        
        case Let(x, e1, e2) => {
            val (v1, store1) = evalExpr(e1, env, store)  // eval e1
            val env2 = env + (x -> v1) // create a new extended env
            evalExpr(e2, env2, store1) // eval e2 under that.
        }
    
        case FunDef(x, e) => {
            (Closure(x, e, env), store) // Return a closure with the current enviroment.
        }
        
        case FunCall(e1, e2) => {
            val (v1, store1) = evalExpr(e1, env, store)
            val (v2, store2) = evalExpr(e2, env, store1)
            v1 match {
                case Closure(x, closure_ex, closed_env) => {
                    // First extend closed_env by binding x to v2
                    val new_env = closed_env + ( x -> v2)
                    // Evaluate the body of the closure under the extended environment.
                    evalExpr(closure_ex, new_env, store2)
                }
                case _ => throw new IllegalArgumentException(s"Function call error: expression $e1 does not evaluate to a closure")
            }
        }
        
        
        
        case AssignVar(x, e) => { // x is a string -- name of identifier and e is Expr -- RHS of assignment
            val (v1, store1) = evalExpr(e, env, store) // First evaluate e
            val v2 =  if (env contains x)  // Next, check x from the current environment
                        env(x)
                      else 
                        throw new IllegalArgumentException(s"Undefined identifier $x")// Trying to assign to an undeclared identifier
            v2 match {
                case Reference(j) => { // x better be a reference in the current env.
                    val store3 = assignToCell(store1, j, v1) // assign to cell function in ImmutableStore API
                    (v1, store3) 
                }
                case _ => throw new IllegalArgumentException(s"AssignVar applied to argument that is not a mutable var")
                
            }
        }
        
        case LetVar(x, e1, e2) => { // let var x = e1 in e2 
            // This is the same treatment as let x = newref(e1) in e2 in ExplicitRef Language.
            val (v1, store1) = evalExpr(e1, env, store) // evaluate e1
            val (store2, j) = createNewCell(store1, v1) // create a new cell corresponding to the value of e1
            val newEnv = env + (x -> Reference(j)) // update the environment
            evalExpr(e2, newEnv, store2) // evaluatet e2 with the new environment and the new store.
        }
        
    }

}

def evalProgram(p: Program) = p match {
        case TopLevel(e) => { 
            // Start with empty environment and empty store
            val (v1, s1) = evalExpr(e, Map(), new ImmutableStore(0, Map()))
            v1
        }
}
    


defined function evalExpr
defined function evalProgram

In [8]:
/* let var x = 10 in 
    let dummy = AssignVar(x, 20) in 
        x
        */
val x = Ident("x")
val e1 = Let("dummy", AssignVar("x", Const(20)), x)
val e2 = LetVar("x", Const(10), e1)
val prog = TopLevel(e2)

println(s"Result = ${evalProgram(prog)}")


Result = NumValue(20.0)


x: Ident = Ident("x")
e1: Let = Let(dummy,AssignVar(x,Const(20.0)),Ident(x))
e2: LetVar = LetVar(x,Const(10.0),Let(dummy,AssignVar(x,Const(20.0)),Ident(x)))
prog: TopLevel = TopLevel(LetVar(x,Const(10.0),Let(dummy,AssignVar(x,Const(20.0)),Ident(x))))

In [9]:
/*~~~ 
let var x = 10 in 
   let g = function (y) 
            x
        in 
       let dummy = AssignVar(x, 20) in 
            g (dummy)
~~~*/
val g = Ident("g")
val dummy = Ident("dummy")
val x = Ident("x")

val e1 = FunCall(g, dummy)
val e2 = Let("dummy", AssignVar("x", Const(20)), e1)
val gdef = FunDef("y", x)
val e3 = Let("g", gdef, e2)
val e4 = LetVar("x",  Const(10), e3)
val prog = TopLevel(e4)

println(s"Result = ${evalProgram(prog)}")



Result = NumValue(20.0)


g: Ident = Ident("g")
dummy: Ident = Ident("dummy")
x: Ident = Ident("x")
e1: FunCall = FunCall(Ident(g),Ident(dummy))
e2: Let = Let(dummy,AssignVar(x,Const(20.0)),FunCall(Ident(g),Ident(dummy)))
gdef: FunDef = FunDef(y,Ident(x))
e3: Let = Let(g,FunDef(y,Ident(x)),Let(dummy,AssignVar(x,Const(20.0)),FunCall(Ident(g),Ident(dummy))))
e4: LetVar = LetVar(x,Const(10.0),Let(g,FunDef(y,Ident(x)),Let(dummy,AssignVar(x,Const(20.0)),FunCall(Ident(g),Ident(dummy)))))
prog: TopLevel = TopLevel(LetVar(x,Const(10.0),Let(g,FunDef(y,Ident(x)),Let(dummy,AssignVar(x,Const(20.0)),FunCall(Ident(g),Ident(dummy))))))

In [11]:
/*---
let var f = function (x) x + 10 in 
  let g = function (y) y - 5 in 
    let d = f(10) in 
       let dummy = AssignVar(f, g) in 
          d - f(10)
          */
val d = Ident("d")
val f = Ident("f")
val g = Ident("g")
val x = Ident("x")
val y = Ident("y")

val e1 = Minus(d, FunCall(f, Const(10)))
val e2 = Let("dummy", AssignVar("f", g), e1)
val e3 = Let("d", FunCall(f, Const(10)), e2)
val gdef = FunDef("y", Minus(y, Const(5)))
val e4 = Let("g", gdef, e3)
val fdef = FunDef("x", Plus(x, Const(10)))
val e5 = LetVar("f", fdef, e4)
val prog = TopLevel(e5)
println(s"Result = ${evalProgram(prog)}")

Result = NumValue(15.0)


d: Ident = Ident("d")
f: Ident = Ident("f")
g: Ident = Ident("g")
x: Ident = Ident("x")
y: Ident = Ident("y")
e1: Minus = Minus(Ident(d),FunCall(Ident(f),Const(10.0)))
e2: Let = Let(dummy,AssignVar(f,Ident(g)),Minus(Ident(d),FunCall(Ident(f),Const(10.0))))
e3: Let = Let(d,FunCall(Ident(f),Const(10.0)),Let(dummy,AssignVar(f,Ident(g)),Minus(Ident(d),FunCall(Ident(f),Const(10.0)))))
gdef: FunDef = FunDef(y,Minus(Ident(y),Const(5.0)))
e4: Let = Let(g,FunDef(y,Minus(Ident(y),Const(5.0))),Let(d,FunCall(Ident(f),Const(10.0)),Let(dummy,AssignVar(f,Ident(g)),Minus(Ident(d),FunCall(Ident(f),Const(10.0))))))
fdef: FunDef = FunDef(x,Plus(Ident(x),Const(10.0)))
e5: LetVar = LetVar(f,FunDef(x,Plus(Ident(x),Const(10.0))),Let(g,FunDef(y,Minus(Ident(y),Const(5.0))),Let(d,FunCall(Ident(f),Const(10.0)),Let(dummy,AssignVar(f,Ident(g)),Minus(Ident(d),FunCall(Ident(f),Const(10.0)))))))
prog: TopLevel = TopLevel(LetVar(f,FunDef(x,Plus(Ident(x),Const(10.0))),Let(g,FunDef(y,Minus(Ident(y),Const(5.0))),Let(d,Fu